# Capítulo 5 - Parte 1: Q-Learning, SARSA e Expected-SARSA


Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no link abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap05/cap05-main-1.ipynb)

## 1. Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg freeglut3-dev xvfb

    !pip install gym==0.23.1

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'

    from util.notebook import display_videos_from_path

### Imports Gerais

In [ ]:
import gym
import numpy as np

In [ ]:
from util.experiments import test_greedy_Q_policy, repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path

In [ ]:
# define se os algoritmos vão imprimir dados do treinamento
VERBOSE = True

## 1. Q-Learning

Veremos aqui, vários algoritmos da família *Temporal-Difference Learning* de 1 passo. Todos eles se baseiam na em uma estimativa da função $Q(s,a)$. Porém, diferente dos Monte Carlo, eles apresendem a cada passo de interação com o episódio.

Ao invés do retorno parcial real $G_t$, todos usam uma estimativa *bootstraped* baseada nas equações de Bellamn. 

Após uma experiência $(s,a,r,s')$, eles atualizam $Q$ assim:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \times (r + \gamma V(s') - Q(s,a)) 
$$

O método **Q-Learning** tem como característica a suposição de que, a partir de s', será executada a ação "gulosa". Assim, ele assume que, para $s'$ não-terminal:

$$
V(s') = max_{a'}(Q(s',a'))
$$

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def epsilon_greedy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])

In [ ]:
# Algoritmo Q-learning, online learning (TD-learning)
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_qlearning(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low = -1.0, high = 0.0,
                          size = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa um episódio completo, fazendo atualizações na Q-table
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q

In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 5000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo Q-Learning
rewards1, qtable1 = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON, render=False)
clear_output()

print("Últimos resultados: media =", np.mean(rewards1[-20:]), ", desvio padrao =", np.std(rewards1[-20:]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards1, r_max, None)

In [ ]:
test_greedy_Q_policy(env, qtable1, 1, render=True)
#test_greedy_Q_policy(env, qtable1, 1, render=True, recorded_video_folder='./videos-qlearning')

In [ ]:
#display_videos_from_path('./videos-qlearning')

## 2. SARSA

O **SARSA** é um algoritmo semelhante ao Q-Learning. Porém, ele usa um valor de $V(s')$ baseado na próxima ação ($a'$) realmente escolhida.

Em outras palavras, o Q-Learning:
- usa os dados $(s,a,r,s')$ para fazer uma atualização no $Q(s,a)$
- assume que o valor do próximo estado é (guloso) baseado na melhor ação possível: $V(s') = max_{a'}{Q(s', a')}$

Porém, o SARSA:
- usa os dados $(s,a,r,s',a')$ para fazer uma atualização no $Q(s,a)$
- assume que o valor do próximo estado é o resultado da verdadeira próxima ação $a'$: $V(s') = Q(s', a')$

Crie o código do SARSA abaixo, com base no código do Q-Learning. Dicas:
- para atualizar o $Q$, você precisa desta 4 variáveis coerentemente mantidas: `state, action, next_state, next_action`
- escolha a "próxima ação" imediatamente após cada passo (e logo após o reset) e antes da atualização do $Q$

In [ ]:
# Algoritmo SARSA, online learning (TD-learning)
# Atenção: os espaços de estados e de ações precisam ser discretos, dados por valores inteiros
def run_sarsa(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low = -1.0, high = 0.0,
                          size = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        # IMPLEMENTE A PARTIR DAQUI (removendo a exceção)

        # remova
        raise Exception("Not implemented")

        # ATÉ AQUI....

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q

Se não conseguir, descomente o código abaixo para prosseguir:

In [ ]:
#from cap05.nstep_sarsa import run_nstep_sarsa
#run_sarsa = lambda env, episodes, lr, gamma, eps, render=False : run_nstep_sarsa(env, episodes, 1, lr, gamma, eps, render=render)

In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 5000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

# Roda o algoritmo SARSA
rewards2, qtable2 = run_sarsa(env, EPISODES, LR, GAMMA, EPSILON, render=False)
clear_output()

print("Últimos resultados: media =", np.mean(rewards2[-20:]), ", desvio padrao =", np.std(rewards2[-20:]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards2, r_max, None)

In [ ]:
test_greedy_Q_policy(env, qtable2, 1, render=False)

## 3. Expected-SARSA

O Expected-SARSA tem semelhanças com os dois algoritmos vistos antes:
- usa os dados $(s,a,r,s')$ para fazer uma atualização no $Q(s,a)$
- assume que o valor do próximo estado é o valor médio, com base nas probabilidades dadas pela política:
   $$V(s') = \displaystyle\sum_{a'} \pi(a'|s') \times Q(s',a')$$

Para implementá-lo, precisamos das probabilidades da política realizar cada ação.

A função abaixo retorna as probabilidades de cada ação na política *epsilon-greedy*:

In [ ]:
def epsilon_greedy_probs(Q, state, num_actions, epsilon):
    # lista com a probabilidade que todas as ações têm de ser escolhidas nas decisões exploratórias (não-gulosas)
    probs = [ (epsilon / num_actions) ] * num_actions

    # adiciona a probabilidade para a ação 'greedy'
    # em caso de empate, só o menor índice é considerado, mas isso é coerente com a implementação de epsilon_greedy()
    greedy_action = np.argmax(Q[state])
    probs[greedy_action] += (1.0 - epsilon)

    return probs

In [ ]:
# Algoritmo Expected-SARSA
def run_expected_sarsa(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    num_actions = env.action_space.n

    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low = -1.0, high = 0.0,
                          size = (env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
            # exibe/renderiza os passos no ambiente, durante 1 episódio a cada mil e também nos últimos 5 episódios
            if render and (i >= (episodes - 5) or (i+1) % 1000 == 0):
                env.render()

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)

            if done:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor esperado
                p_next_actions = epsilon_greedy_probs(Q, next_state, num_actions, epsilon)
                V_next_state = np.sum( np.asarray(p_next_actions) * Q[next_state] )

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            Qnew = (reward + gamma * V_next_state) 
            Q[state,action] = Q[state,action] + lr * (Qnew - Q[state,action])

            sum_rewards += reward
            state = next_state

        # salva o retorno do episódio que encerrou
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q


In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 7000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1

env = gym.make(ENV_NAME)

rewards3, qtable3 = run_expected_sarsa(env, EPISODES, LR, GAMMA, EPSILON, render=False)
clear_output()

print("Últimos resultados: media =", np.mean(rewards3[-20:]), ", desvio padrao =", np.std(rewards3[-20:]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards3, r_max, None)

In [ ]:
test_greedy_Q_policy(env, qtable3, 1, render=True)

## 4. Experimentos

In [ ]:
# parâmetros para todos os algoritmos (não é o ideal)
LR = 0.1
GAMMA = 0.95
EPSILON = 0.05

# silencia os algoritmos
VERBOSE = False

# repetições de cada algoritmo
RUNS = 5

# para carregar automaticamente os resultados de simulações já executadas
auto_load = True

### Taxi-v3 (discreto)

In [ ]:
environment = gym.make("Taxi-v3")
NUM_EPISODES = 5_000

results_t = []

results_t.append( repeated_exec(RUNS, f"Q-Learning", run_qlearning, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()
results_t.append( repeated_exec(RUNS, f"Sarsa", run_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()
results_t.append( repeated_exec(RUNS, f"Exp-Sarsa", run_expected_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()

In [ ]:
plot_multiple_results(results_t, cumulative=False, x_log_scale=False, window=50)

### FrozenLake-v1

In [ ]:
environment = gym.make("FrozenLake-v1")
NUM_EPISODES = 8_000

results_f = []

results_f.append( repeated_exec(RUNS, f"Q-Learning", run_qlearning, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()
results_f.append( repeated_exec(RUNS, f"Sarsa", run_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()
results_f.append( repeated_exec(RUNS, f"Exp-Sarsa", run_expected_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()

In [ ]:
plot_multiple_results(results_f, cumulative=False, x_log_scale=False, window=100)

### Cliff Walking

In [ ]:
environment = gym.make("CliffWalking-v0")
NUM_EPISODES = 3_000

results_c = []

results_c.append( repeated_exec(RUNS, f"Q-Learning", run_qlearning, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()
results_c.append( repeated_exec(RUNS, f"Sarsa", run_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()
results_c.append( repeated_exec(RUNS, f"Exp-Sarsa", run_expected_sarsa, environment, NUM_EPISODES, LR, GAMMA, EPSILON, auto_load=auto_load) )
clear_output()

In [ ]:
plot_multiple_results(results_c, cumulative=False, x_log_scale=False, window=100)